## Засоби підготовки та аналізу даних
#### Корнієнко Андрій, ФБ-32. Лабораторна робота №2

Імпортуємо бібліотеки

In [17]:
import pandas as pd

In [18]:
import urllib.request
from datetime import datetime, date
import os
import csv

Будуємо фундамент програми

In [19]:
url_data=('https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={''}&year1=1981&year2=2024&type=Mean')
dir = 'CSV_Files'
    
indexes = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 15, 19: 16, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
if not os.path.exists(dir):
    os.makedirs(dir)

Створення головних функцій, а саме: конструктор для посилання, чеккер на наявність файлу у директорії, доунлоддер, апдейтер, компілятор ( згідно завданню є процедурою, що також змінює індекси областей, "main()")

In [20]:
def construct_url(province_id):
    return url_data.format(province_id)

def file_exists(directory, file_prefix):
    for file_name in os.listdir(directory):
        if file_name.startswith(file_prefix):
            return True
    return False

def download_file(url, file_path):
    with urllib.request.urlopen(url) as response:
        text = response.read()
        with open(file_path, 'wb') as file:
            file.write(text)

def update_file_if_needed(url, file_path):
    with urllib.request.urlopen(url) as response:
        new_text = response.read()
        with open(file_path, 'rb') as file:
            existing_text = file.read()
        if existing_text != new_text:
            with open(file_path, 'wb') as file:
                file.write(new_text)
            print('Файл оновлено:', file_path)
        else:
            print('Файл з останнім оновленням вже завантажено:', file_path)

def main():
    for province_id, index in indexes.items():
        file_prefix = 'NOAA_' + str(index) + '_'
        file_already_exists = file_exists(dir, file_prefix)

        if not file_already_exists:
            url = construct_url(province_id)
            date_and_time_time = datetime.now().strftime("%Y%m%d%H%M%S")
            file_name = file_prefix + date_and_time_time + '.csv'
            file_path = os.path.join(dir, file_name)
            download_file(url, file_path)
            print('Файл встановленно:', file_name)
        else:
            file_path = next((os.path.join(dir, file_name) for file_name in os.listdir(dir) if file_name.startswith(file_prefix)), None)
            if file_path:
                update_file_if_needed(construct_url(province_id), file_path)

    print('Виконано')

main()

Файл з останнім оновленням вже завантажено: CSV_Files\NOAA_22_20250518203752.csv
Файл з останнім оновленням вже завантажено: CSV_Files\NOAA_24_20250518203753.csv
Файл з останнім оновленням вже завантажено: CSV_Files\NOAA_23_20250518203754.csv
Файл з останнім оновленням вже завантажено: CSV_Files\NOAA_25_20250518203755.csv
Файл з останнім оновленням вже завантажено: CSV_Files\NOAA_3_20250518203757.csv
Файл з останнім оновленням вже завантажено: CSV_Files\NOAA_4_20250518203758.csv
Файл з останнім оновленням вже завантажено: CSV_Files\NOAA_8_20250518203759.csv
Файл з останнім оновленням вже завантажено: CSV_Files\NOAA_19_20250518203800.csv
Файл з останнім оновленням вже завантажено: CSV_Files\NOAA_20_20250518203801.csv
Файл з останнім оновленням вже завантажено: CSV_Files\NOAA_21_20250518203802.csv
Файл з останнім оновленням вже завантажено: CSV_Files\NOAA_9_20250518203805.csv
Файл з останнім оновленням вже завантажено: CSV_Files\NOAA_10_20250518203806.csv
Файл з останнім оновленням вже з

Створюємо потрібні нам змінні 

In [21]:
output_path = "united.csv"

column_names = ["Year", "Week", "SMN", "SMT", "VCI", "TCI", "VHI", "Area"]
combined_data = pd.DataFrame(columns=column_names)

filenames = os.listdir(dir)

for filename in filenames:
    if not filename.endswith(".csv"):
        continue

    file_path = os.path.join(dir, filename)

    df = pd.read_csv(file_path, skiprows=2, names=column_names)
    df["Year"] = df["Year"].str.replace('<tt><pre>', '').str.replace('</pre></tt>', '')

    region_id = int(filename.split('_')[1])
    df["Area"] = region_id

    df = df.drop(df.loc[df['VHI'] == -1].index).dropna()

    combined_data = pd.concat([combined_data, df], ignore_index=True)

combined_data.to_csv(output_path, index=False)

print(combined_data)

C:\Users\tyb1l\AppData\Local\Temp\ipykernel_20172\1696020492.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_data = pd.concat([combined_data, df], ignore_index=True)


       Year  Week    SMN     SMT    VCI    TCI    VHI Area
0      1982   1.0  0.045  261.12  36.68  41.79  39.23   10
1      1982   2.0  0.041  262.61  36.46  35.10  35.78   10
2      1982   3.0  0.041  263.82  36.49  30.79  33.64   10
3      1982   4.0  0.038  264.71  33.46  28.66  31.06   10
4      1982   5.0  0.034  264.76  29.66  30.50  30.08   10
...     ...   ...    ...     ...    ...    ...    ...  ...
54645  2024  48.0  0.104  270.28  54.76  25.04  39.90    9
54646  2024  49.0  0.091  268.05  49.12  29.95  39.53    9
54647  2024  50.0  0.083  266.38  47.79  31.17  39.48    9
54648  2024  51.0  0.077  265.11  47.37  29.80  38.58    9
54649  2024  52.0  0.078  265.66  52.23  24.26  38.23    9

[54650 rows x 8 columns]


Ініціюємо датафрейм

In [22]:
df=pd.read_csv('united.csv')

Ряд VHI для області за вказаний рік, пошук екстремумів (min та max), середнє значення та медіана.#

In [23]:
def procedure_1(dataframe, index, year):
    vhi = dataframe[(dataframe["Area"] == index) & (dataframe["Year"] == year)]['VHI']
    vhi_max = vhi.max()
    vhi_min = vhi.min()
    vhi_mean = vhi.mean()     # Среднее значение
    vhi_median = vhi.median() # Медиана
    vhi_row = ', '.join([str(val) for val in vhi.tolist()])
    return (
        f"Ряд VHI для області за вказаний рік: {vhi_row}.\n"
        f"Мінімальний показник VHI для області з індексом {index} у {year} році складав {vhi_min}. \n"
        f"Максимальний показник VHI для області з індексом {index} у {year} році складав {vhi_max}.\n"
        f"Середнє значення VHI: {vhi_mean:.2f}, медіана: {vhi_median}."
    )

Тест:

In [24]:
index = 7
year = 1993
print(procedure_1(df, index, year))

Ряд VHI для області за вказаний рік: 41.49, 41.02, 38.27, 37.4, 38.0, 36.27, 34.85, 34.19, 33.52, 33.83, 34.49, 35.82, 36.66, 36.06, 36.48, 38.12, 41.9, 48.59, 54.24, 61.19, 67.96, 70.62, 71.54, 73.61, 76.33, 75.15, 69.78, 63.53, 58.93, 54.9, 51.89, 48.95, 49.38, 51.06, 52.09, 51.45, 49.46, 48.51, 47.07, 43.51, 39.42, 37.1, 34.77, 31.56, 29.51, 29.74, 29.7, 28.41, 27.52, 27.75, 28.31, 30.66.
Мінімальний показник VHI для області з індексом 7 у 1993 році складав 27.52. 
Максимальний показник VHI для області з індексом 7 у 1993 році складав 76.33.
Середнє значення VHI: 45.05, медіана: 40.22.


Ряд VHI за вказаний діапазон років для вказаних областей;

In [25]:
def procedure_2(dataframe, years_start, years_end, indexes):
    years = range(years_start, years_end)
    vhi_by_index = {}
    for index in indexes:
        filtered_df = dataframe[(dataframe["Area"] == index) & (dataframe["Year"].isin(years))]
        vhi_by_index[index] = filtered_df["VHI"].tolist()

    for index, vhi in vhi_by_index.items():
        print(f"Ряд VHI для області з індексом {index} за вказаний діапазон років: {', '.join([str(val) for val in vhi])}")

Тест:

In [26]:
starting_year = 1993
ending_year = 1995
province_indexes = [7, 2, 8]
procedure_2(df, starting_year, ending_year, province_indexes)

Ряд VHI для області з індексом 7 за вказаний діапазон років: 41.49, 41.02, 38.27, 37.4, 38.0, 36.27, 34.85, 34.19, 33.52, 33.83, 34.49, 35.82, 36.66, 36.06, 36.48, 38.12, 41.9, 48.59, 54.24, 61.19, 67.96, 70.62, 71.54, 73.61, 76.33, 75.15, 69.78, 63.53, 58.93, 54.9, 51.89, 48.95, 49.38, 51.06, 52.09, 51.45, 49.46, 48.51, 47.07, 43.51, 39.42, 37.1, 34.77, 31.56, 29.51, 29.74, 29.7, 28.41, 27.52, 27.75, 28.31, 30.66, 39.23, 46.16, 51.89, 52.1, 53.06, 52.14, 51.12, 48.9, 49.26, 48.63, 46.62, 42.8, 39.0, 36.75, 36.57, 36.94, 42.73, 52.59, 61.01, 66.2, 71.16, 75.04, 79.29, 83.26, 83.28, 79.24, 73.35, 65.67, 58.07, 52.21, 48.24, 44.62, 40.92, 37.86, 34.65, 30.25
Ряд VHI для області з індексом 2 за вказаний діапазон років: 33.74, 31.88, 30.67, 32.28, 32.52, 32.95, 33.61, 34.66, 36.74, 38.89, 39.92, 41.68, 42.32, 43.61, 43.59, 40.95, 36.89, 35.71, 38.82, 43.81, 45.92, 46.81, 47.2, 47.19, 48.82, 49.53, 51.3, 54.46, 58.41, 61.69, 62.56, 62.8, 61.14, 58.14, 55.28, 54.13, 51.62, 50.18, 49.95, 49.2

Для всього набору даних виявити роки, протягом яких екстремальні
посухи торкнулися більше вказаного відсотка областей по Україні (20%
областей - 5 областей з 25). Повернути роки, назви областей з
екстремальними посухами та значення VHI;

In [29]:
def procedure_3(dataframe, percent):
    drought_df = dataframe[dataframe["VHI"] < 15]

    yearly_affected_counts = drought_df.groupby("Year")["Area"].nunique()
    total_areas = dataframe["Area"].nunique()
    yearly_percentage_affected = yearly_affected_counts / total_areas

    years_with_extreme_drought = yearly_percentage_affected[yearly_percentage_affected >= percent].index.tolist()

    print("Роки з екстремальною посухою (VHI < 15), що торкнулася більше", int(percent * 100), "% областей:")

    for year in years_with_extreme_drought:
        print(f"\nРік: {year}")
        affected_rows = drought_df[drought_df["Year"] == year]

        grouped = affected_rows.groupby("Area")["VHI"].apply(list)

        for area_id, vhi_values in grouped.items():
            vhi_list_str = ', '.join(f"{v:.2f}" for v in vhi_values)
            print(f" - Область ID: {area_id} | VHI: {vhi_list_str}")


Тест:

In [30]:
procedure_3(df, 0.2)

Роки з екстремальною посухою (VHI < 15), що торкнулася більше 20 % областей:

Рік: 2007
 - Область ID: 7 | VHI: 14.26, 13.39, 13.33, 13.06, 13.13, 12.51, 11.55, 10.88, 11.06, 12.05, 13.84
 - Область ID: 13 | VHI: 11.44, 7.78, 6.44, 6.12, 6.11, 5.94, 6.36, 7.61, 9.45, 11.70
 - Область ID: 14 | VHI: 11.88, 9.21, 7.08, 5.90, 5.52, 5.85, 6.82, 8.86, 11.91
 - Область ID: 20 | VHI: 12.41, 12.23, 12.99, 13.33, 12.88, 12.63, 12.96, 13.48, 14.05, 14.41, 14.81
 - Область ID: 25 | VHI: 14.98, 14.23, 13.79, 13.41, 13.28, 14.36
